This file is something I'm currently messing around with, the point is to
find a good analytical solution for DARD that's reasonably fast.

**Scroll down to the my_dard function for the current implementation**




In [1]:
import numpy as np
from copy import deepcopy
from distance import canon
from _types import Reward
from env import Env, RandomEnv
from tests.toy_env import env, reward
from coverage_dist import get_state_dist, get_action_dist


In [2]:
# manual implementation where we calculate the EV terms as explicit sums via for loops
def dard_slow(reward: Reward, env: Env):
  action_dist = get_action_dist(env)

  exp_term1 = np.zeros(env.n_s)
  for s_prime in range(env.n_s):
    for A, A_prob in enumerate(action_dist):
      for S_double, S_double_prob in enumerate(env.transition_dist[s_prime, A]):
        prob = A_prob * S_double_prob
        exp_term1[s_prime] += prob * env.discount * reward[s_prime, A, S_double]
  exp_term1 = exp_term1[None, None, :]

  exp_term2 = np.zeros(env.n_s)
  for s in range(env.n_s):
    for A, A_prob in enumerate(action_dist):
      for S_prime, S_prime_prob in enumerate(env.transition_dist[s, A]):
        prob = A_prob * S_prime_prob
        exp_term2[s] += prob * reward[s, A, S_prime]
  exp_term2 = exp_term2[:, None, None]

  exp_term3 = np.zeros((env.n_s, env.n_s))
  for s in range(env.n_s):
    for s_prime in range(env.n_s):
      for A, A_prob in enumerate(action_dist):
        for S_prime, S_prime_prob in enumerate(env.transition_dist[s, A]):
          for S_double, S_double_prob in enumerate(env.transition_dist[s_prime, A]):
            prob = A_prob * S_prime_prob * S_double_prob
            exp_term3[s, s_prime] += prob * env.discount * reward[S_prime, A, S_double]
  exp_term3 = exp_term3[:, None, :]

  return exp_term1, exp_term2, exp_term3

  return reward + exp_term1 - exp_term2 - exp_term3

In [3]:
def gradient(potential, gamma):
  return (
      gamma * potential[..., None, None, :] - potential[..., :, None, None]
  )


In [4]:
# from the file Joar sent over
def dard_reference(rewards, weights, gamma):
  """Compute the DARD transformation of a given reward function."""
  weighted_rewards = rewards * weights
  outflow = weighted_rewards.sum((-2, -1))
  # outflow contains state distribution, which we want to remove:
  potential = outflow / weights.sum(axis=(1, 2))
  joint_state_probs = weights.sum(axis=1)
  # next_state_probs[s, s'] = P(s' | s)
  next_state_probs = joint_state_probs / joint_state_probs.sum(
      axis=1, keepdims=True
  )
  action_probs = weights.sum(axis=(0, 2))
  # I'm calling this an "offset" in analogy to EPIC, but note that it's not
  # a constant!
  offset = (
      rewards[None, :, :, None, :]
      * next_state_probs[:, :, None, None, None]
      * action_probs[None, None, :, None, None]
      * next_state_probs[None, None, None, :, :]
  ).sum(axis=(1, 2, 4))

  return gamma * potential[..., None, None, :], potential[..., :, None, None], gamma * offset[:, None, :]

  return rewards + gradient(potential, gamma) - gamma * offset[:, None, :]



In [5]:
# Let the fuckery commence!

# 1 what the fuck are weights?
slow_result = dard_slow(reward, env)
D_s = get_state_dist(env)
D_a = get_action_dist(env)
weights = D_s[:, None, None] * D_a[None, :, None] * D_s[None, None, :]
ref_result = dard_reference(reward, weights, env.discount)
slow_result, ref_result

((array([[[0.45, 0.  ]]]),
  array([[[0.5]],
  
         [[0. ]]]),
  array([[[0.  , 0.  ]],
  
         [[0.45, 0.  ]]])),
 (array([[[0.225, 0.   ]]]),
  array([[[0.25]],
  
         [[0.  ]]]),
  array([[[0.1125, 0.1125]],
  
         [[0.1125, 0.1125]]])))

In [6]:
env2 = RandomEnv(3, 2)
reward2 = np.zeros((3, 2, 3))
reward2[0, 1, 1] = 1.0

D_s = get_state_dist(env2)
D_a = get_action_dist(env2)
weights = D_s[:, None, None] * D_a[None, :, None] * D_s[None, None, :]
dard_slow(reward2, env2), dard_reference(reward2, weights, env.discount)

((array([[[0.1023023, 0.       , 0.       ]]]),
  array([[[0.11366922]],
  
         [[0.        ]],
  
         [[0.        ]]]),
  array([[[0.06387757, 0.12307836, 0.08056726]],
  
         [[0.03838355, 0.07395685, 0.04841226]],
  
         [[0.03305695, 0.06369364, 0.04169394]]])),
 (array([[[0.11059644, 0.        , 0.        ]]]),
  array([[[0.12288494]],
  
         [[0.        ]],
  
         [[0.        ]]]),
  array([[[0.04485786, 0.04485786, 0.04485786]],
  
         [[0.04485786, 0.04485786, 0.04485786]],
  
         [[0.04485786, 0.04485786, 0.04485786]]])))

In [7]:
def my_dard(reward: Reward, env: Env):
  s = np.ones(env.n_s) # [s]
  s_prime = np.ones(env.n_s) # [s_prime]
  A = get_action_dist(env) # [A]

  # Given a value of s, what are the probabilities of different A-S_prime combos
  S_prime = env.transition_dist * s[:, None, None] * A[None, :, None] # [s, A, S_prime]

  # Given a value of s_prime, what are the probabilities of different A-S_double combos
  S_double = env.transition_dist * s_prime[:, None, None] * A[None, :, None] # [s_prime, A, S_double]

  # gamma R(s_prime, A, S_double)
  exp1 = env.discount * reward * S_double # [s, a, s_prime]
  exp1 = exp1.sum(axis=(0, 1), keepdims=True) # [1, 1, s_prime]

  # R(s, A, S_prime)
  exp2 = reward * S_prime # [s, a, s_prime]
  exp2 = exp2.sum(axis=(1, 2), keepdims=True) # [s, 1, 1]

  # gamma R(S_prime, A, S_double)
  exp3 = env.discount * reward * ( # [s, a, s_prime]
    S_prime.sum(axis=(1, 2), keepdims=True) *
    A[None, :, None] *
    S_double.sum(axis=(0, 1), keepdims=True)
  )
  exp3 = exp3.sum(axis=1, keepdims=True) # [s, 1, s_prime]

  return exp1, exp2, exp3

  return reward + exp1 - exp2 - exp3

In [8]:
# should output
# ((array([[[0.45, 0.  ]]]),
#   array([[[0.5]],
#          [[0. ]]]),
#   array([[[0.  , 0.  ]],
#          [[0.45, 0.  ]]]))
my_dard(reward, env)

(array([[[0.  , 0.45]]]),
 array([[[0.5]],
 
        [[0. ]]]),
 array([[[0.  , 0.45]],
 
        [[0.  , 0.  ]]]))

# IDEA: sample expectation instead of the analytical solution if that doesn't work